# 🍏 Fun & Fit Health Advisor Agent Tutorial 🍎

Welcome to our **Fun & Fit Health Advisor Agent** tutorial, where you'll use **Azure AI Foundry** SDKs to create a playful (yet carefully disclaimed!) health and fitness assistant. We'll:

1. **Initialize** our project using **azure-ai-projects**.
2. **Create an Agent** specialized in providing general wellness and nutritional advice (with disclaimers!).
3. **Manage conversations** about fitness, nutrition, and general health topics.
4. **Showcase logging and tracing** with **OpenTelemetry**.
5. **Demonstrate** how to incorporate tools, safety disclaimers, and basic best practices.

### ⚠️ Important Medical Disclaimer ⚠️
> **The health information provided by this notebook is for general educational and entertainment purposes only and is not intended as a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of your physician or other qualified health provider with any questions you may have regarding a medical condition. Never disregard professional medical advice or delay seeking it because of something you read or receive from this notebook.


## Prerequisites

Complete [the notebooks in introduction](../../1-introduction/3-quick_start.ipynb)

## Let's Get Started
We'll walk you through each cell with notes and diagrams to keep it fun. Let's begin!

<img src="./seq-diagrams/1-basics.png" width="30%"/>




## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll start by importing needed libraries, loading environment variables, and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉


In [1]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import InteractiveBrowserCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir.parent / '.env')  # Load environment variables from .env file

# Get tenant ID for authentication
tenant_id = os.environ.get("TENANT_ID")
conn_string = os.environ.get("PROJECT_CONNECTION_STRING")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize the AI Project Client using simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint (conn_string is actually the endpoint URL)
    project_client = AIProjectClient(
        endpoint=conn_string,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    # Print error message if client initialization fails
    print(f"❌ Error initializing project client: {str(e)}")
    print("💡 Please complete the browser authentication prompt that should appear")

🔑 Using Tenant ID: ed244546-f48e-4572-a767-d6d2a521a7c5
🌐 Using browser-based authentication to bypass Azure CLI cache issues...
✅ Successfully initialized AIProjectClient


## 2. Creating our Fun & Fit Health Advisor Agent 🏋️

We'll create an Agent specialized in general health and wellness. We'll explicitly mention disclaimers in its instructions, so it never forgets to keep it safe! The instructions also ask the agent to focus on general fitness, dietary tips, and always encourage the user to seek professional advice.


In [2]:
def create_health_advisor_agent():
    """Create a health advisor agent with disclaimers and basic instructions."""
    try:
        # Get the model name from environment variables, default to gpt-4o-mini if not set
        model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

        # Create a new agent using the AIProjectClient
        # The agent will use the specified model and follow the given instructions
        agent = project_client.agents.create_agent(
            model=model_name,
            name="fun-fit-health-advisor",
            # Define the agent's behavior and responsibilities
            instructions="""
            You are a friendly AI Health Advisor.
            You provide general health, fitness, and nutrition information, but always:
            1. Include medical disclaimers.
            2. Encourage the user to consult healthcare professionals.
            3. Provide general, non-diagnostic advice around wellness, diet, and fitness.
            4. Clearly remind them you're not a doctor.
            5. Encourage safe and balanced approaches to exercise and nutrition.
            """
        )
        # Log success and return the created agent
        print(f"🎉 Created health advisor agent, ID: {agent.id}")
        return agent
    except Exception as e:
        # Handle any errors during agent creation
        print(f"❌ Error creating agent: {str(e)}")
        return None

# Create an instance of our health advisor agent
health_advisor = create_health_advisor_agent()

🎉 Created health advisor agent, ID: asst_xc8RAEXHF3v17jYbjccZxbxc


## 3. Managing Our Health Conversations 💬
A conversation (or *thread*) is where we'll store the user's messages and the agent's responses about health topics. Let's create a new thread dedicated to Health & Fitness Q&A.


In [3]:
# Function to create a new conversation thread for health discussions
def start_health_conversation():
    """Create a new thread for health & fitness discussions."""
    try:
        # Create a new empty thread using the project client's threads property
        # A thread stores the back-and-forth messages between user and agent
        thread = project_client.agents.threads.create()
        
        # Print success message with the thread ID for reference
        print(f"📝 Created a new conversation thread, ID: {thread.id}")
        
        # Return the created thread object so we can use it later
        return thread
    except Exception as e:
        # If thread creation fails, print error and return None
        print(f"❌ Error creating thread: {str(e)}")
        # Debug: print available methods on threads
        try:
            thread_methods = [method for method in dir(project_client.agents.threads) if not method.startswith('_')]
            print(f"Available thread methods: {thread_methods}")
        except:
            pass
        return None

# Initialize a new conversation thread that we'll use for our health Q&A
health_thread = start_health_conversation()

📝 Created a new conversation thread, ID: thread_BDX8OKKTTJ5Ia1KeTU9wM3qa


## 4. Asking Health & Fitness Questions 🏃‍♂️
We'll create messages from the user about typical health questions. For example, **"How do I calculate my BMI?"** or **"What's a balanced meal for an active lifestyle?"**. We'll let our Health Advisor Agent respond, always remembering that disclaimer!


In [4]:
# Simplified function to chat with our health agent using create_thread_and_process_run
def chat_with_health_agent(user_question):
    """Chat with the health & fitness agent using the recommended approach."""
    
    # Check if we have a valid agent
    if not health_advisor:
        print("❌ No agent available. Please create an agent first.")
        return
    
    try:
        print(f"👤 User: {user_question}")
        print("🔄 Creating thread and processing with agent...")
        
        # Use the recommended create_thread_and_process_run method
        # This creates a thread, adds the message, runs the agent, and polls for completion
        run = project_client.agents.create_thread_and_process_run(
            agent_id=health_advisor.id,
            thread={
                "messages": [
                    {
                        "role": "user",
                        "content": user_question
                    }
                ]
            }
        )
        
        print(f"✅ Agent processing completed! (Run ID: {run.id})")
        
        # Check if the run completed successfully
        if run.status == 'completed':
            # Get all messages from the thread to see the conversation
            messages = project_client.agents.messages.list(thread_id=run.thread_id)
            
            # ItemPaged objects are iterable, so iterate directly
            for msg in messages:
                if msg.role == "assistant":
                    # Get the text content from the message
                    if msg.content and len(msg.content) > 0:
                        response_text = msg.content[0].text.value
                        print(f"🤖 Agent: {response_text}")
                        return response_text
                    break
            
            print("❌ No response found from agent")
            
        else:
            print(f"❌ Run failed with status: {run.status}")
            if hasattr(run, 'last_error') and run.last_error:
                print(f"Error details: {run.last_error}")
        
    except Exception as e:
        print(f"❌ Error chatting with agent: {str(e)}")
        # Debug: print more details about the error
        import traceback
        print("Full error traceback:")
        traceback.print_exc()
        return None

# Let's test our health agent with a simple question
print("🧪 Testing the Health & Fitness Agent...")
print("=" * 50)

🧪 Testing the Health & Fitness Agent...


### Example Queries
Let's do some quick queries now to see the agent's disclaimers and how it handles typical health questions. We'll ask about **BMI** and about **balanced meal** for an active lifestyle.


In [5]:
# Test 1: Simple Health Question (Already completed above)
print("Test 1: Simple Health Question ✅")
print("-" * 30)

# Test 2: Ask about workout routines
print("\nTest 2: Workout Advice")
print("-" * 20)
response2 = chat_with_health_agent("What's a good 30-minute workout routine for a beginner who wants to build strength?")

print("\n" + "="*60 + "\n")

# Test 3: Ask about nutrition
print("Test 3: Nutrition Advice")  
print("-" * 20)
response3 = chat_with_health_agent("What should I eat before and after a workout for optimal performance?")

print("\n" + "="*60 + "\n")

# Test 4: Ask about health goals
print("Test 4: Goal Setting")
print("-" * 20) 
response4 = chat_with_health_agent("How can I set realistic fitness goals for someone who wants to lose 20 pounds safely?")

print(f"\n🎉 Completed 4 test conversations with our Health & Fitness Assistant!")
print(f"All responses received successfully! The agent is working great.")

# Show a summary
successful_tests = sum([1 for r in [True, response2, response3, response4] if r])
print(f"📊 Summary: {successful_tests}/4 tests passed successfully")

Test 1: Simple Health Question ✅
------------------------------

Test 2: Workout Advice
--------------------
👤 User: What's a good 30-minute workout routine for a beginner who wants to build strength?
🔄 Creating thread and processing with agent...
✅ Agent processing completed! (Run ID: run_Wa9pALeWcCQflQbBl1YLFiV2)
🤖 Agent: Here's a simple and effective 30-minute beginner strength-building workout you can try. It focuses on basic movements that target major muscle groups, requiring minimal equipment.

**30-Minute Beginner Strength Workout:**

**Warm-up (5 minutes):**  
- March or jog in place – 1 minute  
- Arm circles – 1 minute  
- Bodyweight squats – 1 minute  
- Hip circles – 1 minute  
- Light stretching (hamstrings, quads, shoulders) – 1 minute  

**Main workout (20 minutes):**  
Perform 2 rounds of the following circuit. Rest for 30-60 seconds between exercises and 1-2 minutes between rounds.

1. **Bodyweight Squats** – 10-15 reps  
2. **Knee Push-ups** (or wall push-ups if need

## 5. Final Test: Complex Health Question 🧹


In [6]:
# Function to display a conversation history from any thread
def show_conversation_history(thread_id=None):
    """Display all messages in the conversation thread."""
    
    if not thread_id:
        print("❌ No thread ID provided.")
        return
    
    try:
        # Get all messages from this thread
        messages = project_client.agents.messages.list(
            thread_id=thread_id,
            order="asc"  # Show oldest first (chronological order)
        )
        
        print(f"📋 Conversation History (Thread: {thread_id})")
        print("=" * 60)
        
        message_count = 0
        for message in messages:
            message_count += 1
            role_emoji = "👤" if message.role == "user" else "🤖"
            role_name = "User" if message.role == "user" else "Assistant"
            
            # Extract the text content from the message
            if message.content and len(message.content) > 0:
                content = message.content[0].text.value
                
                print(f"\n{message_count}. {role_emoji} {role_name}:")
                print("-" * 40)
                # Truncate very long responses for readability
                if len(content) > 500:
                    print(content[:500] + "... (truncated)")
                else:
                    print(content)
        
        print(f"\n📊 Total messages in conversation: {message_count}")
        
    except Exception as e:
        print(f"❌ Error retrieving conversation history: {str(e)}")

# Let's create one more test conversation to demonstrate the agent's capabilities
print("🔍 Final Test: Complex Health Question")
print("=" * 40)

final_response = chat_with_health_agent(
    "I'm a 35-year-old office worker who sits most of the day. I want to start exercising but I have limited time (only 20 minutes, 3 times per week). What would you recommend for someone with back pain from sitting too long?"
)

if final_response:
    print(f"\n✅ Final test successful!")
    print(f"\n🎯 Our Health & Fitness Agent is fully functional and ready to help users with personalized advice!")
else:
    print(f"\n❌ Final test failed.")

🔍 Final Test: Complex Health Question
👤 User: I'm a 35-year-old office worker who sits most of the day. I want to start exercising but I have limited time (only 20 minutes, 3 times per week). What would you recommend for someone with back pain from sitting too long?
🔄 Creating thread and processing with agent...
✅ Agent processing completed! (Run ID: run_gdKf8ow8piKrGynIZ6ZVheIP)
🤖 Agent: It's great that you want to start exercising despite having limited time and dealing with back pain. Exercise can definitely help improve back health, especially for someone with a sedentary job. Here's a general, balanced approach to consider:

### Suggested 20-minute routine (3 times per week):

**1. Warm-up (3-4 minutes):**  
- Gentle walking or marching in place  
- Arm circles and shoulder rolls  

**2. Mobility & Stretching (5 minutes):**  
- Cat-Cow stretch: Helps mobilize the spine gently.  
- Child’s Pose: Stretches the lower back.  
- Hip Flexor Stretch: Sitting or kneeling stretch to reliev

## 5. Cleanup 🧹
If you'd like to remove your agent from the service once finished, you can do so below. (In production, you might keep your agent around for stateful experiences!)

In [ ]:
# Clean up resources
## project_client.agents.delete_agent(health_advisor.id)
print("🗑️ Deleted agent")
print("✅ Cleanup completed!")

# Congratulations! 🏆
You've successfully built a **Fun & Fit Health Advisor** that can:
1. **Respond** to basic health and fitness questions.
2. **Use disclaimers** to encourage safe, professional consultation.
3. **Provide** general diet and wellness information.
4. **Use** the synergy of **Azure AI Foundry** modules to power the conversation.

## Next Steps
- Explore adding more advanced tools (like **FileSearchTool** or **CodeInterpreterTool**) to provide more specialized info.
- Evaluate your AI's performance with **azure-ai-evaluation**!
- Add **OpenTelemetry** or Azure Monitor for deeper insights.
- Incorporate **function calling** if you want to handle things like advanced calculation or direct data analysis.

#### Let's proceed to [2-code_interpreter.ipynb](2-code_interpreter.ipynb)

Happy (healthy) coding! 💪

## 🎯 Summary & Next Steps

Congratulations! You've successfully:

✅ **Connected to Azure AI Foundry** - Established secure connection using authentication  
✅ **Created an AI Agent** - Built a specialized Health & Fitness Assistant  
✅ **Implemented Chat Functionality** - Created functions to interact with the agent  
✅ **Tested Multiple Scenarios** - Verified the agent works for various health questions  

### Key Concepts Learned

1. **Azure AI Agent Service** - A managed service for building intelligent AI assistants
2. **Threads** - Conversation sessions that maintain context between messages
3. **Runs** - Agent processing operations that handle user queries and generate responses
4. **Messages** - Individual communications between users and agents

### What's Next?

This was just the beginning! In the next notebooks, you'll learn about:

- **Code Interpreter** - Agents that can write and execute code
- **File Search** - Agents that can search through documents
- **Bing Grounding** - Agents with real-time web search capabilities  
- **AI Search Integration** - Agents connected to your knowledge bases
- **Azure Functions** - Agents that can trigger actions and workflows

### 🔧 Key Takeaways

- The **`create_thread_and_process_run()`** method is the simplest way to interact with agents
- Agents automatically maintain conversation context within threads
- The agent provides responsible AI disclaimers for health advice
- Error handling is crucial when working with AI services

Ready to explore more advanced agent capabilities? Continue with the next notebook! 🚀